In [33]:
import yfinance as yf
import pandas as pd
import datetime
from datetime import timedelta

ticker='VEMAX'
gfi = yf.Ticker(ticker)
print(gfi.info['shortName'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('display.precision', 2)

last_dt_prev_year = datetime.date(2023, 12, 29)

Vanguard Emerging Markets Stock


In [34]:
tickers=['VTCLX', 'VBTLX', 'VTMSX', 'VIPSX', 'VFIAX', 'VTIAX', 'VMRXX', 'VTMGX', 'VMSXX', 'VEMAX']
print(tickers)

downloaded = yf.download(tickers = tickers, period='1y', interval='1d')
histpx=downloaded['Adj Close'].sort_index(ascending=False)
histpx.index=histpx.index.date

['VTCLX', 'VBTLX', 'VTMSX', 'VIPSX', 'VFIAX', 'VTIAX', 'VMRXX', 'VTMGX', 'VMSXX', 'VEMAX']


[*********************100%%**********************]  10 of 10 completed


In [35]:
tk_2_name_d = {}

def get_return(px, d1, d2):
    print(px[d2]-px[d1])
    
for tk in tickers:
    gfi = yf.Ticker(tk)
    tk_2_name_d[tk] = gfi.info['shortName']
    print(tk, gfi.info['shortName'])

#tk_2_name_d, tk_2_name_d.keys()

VTCLX Vanguard Tax-Managed Capital Ap
VBTLX Vanguard Total Bond Market Inde
VTMSX Vanguard Tax Managed Small Cap 
VIPSX Vanguard Inflation Protected Se
VFIAX Vanguard 500 Index Fd Admiral S
VTIAX Vanguard Total International St
VMRXX Vanguard Cash Reserves Federal 
VTMGX Vanguard Developed Markets Inde
VMSXX Vanguard Municipal Money Market
VEMAX Vanguard Emerging Markets Stock


In [36]:
#histpx = histpx.sort_index(ascending=False)
print(histpx[:20])

            VBTLX  VEMAX   VFIAX  VIPSX  VMRXX  VMSXX   VTCLX  VTIAX  VTMGX  VTMSX
2024-01-22   9.58  32.68  447.71  11.68    NaN    NaN  251.04  30.27  15.05  86.39
2024-01-21    NaN    NaN     NaN    NaN    1.0    1.0     NaN    NaN    NaN    NaN
2024-01-19   9.56  33.02  446.73  11.67    1.0    1.0  250.17  30.29  15.02  84.77
2024-01-18   9.56  32.74  441.27  11.66    1.0    1.0  247.16  30.15  14.97  83.84
2024-01-17   9.58  32.56  437.39  11.66    1.0    1.0  244.96  29.92  14.84  83.07
2024-01-16   9.61  33.09  439.85  11.69    1.0    1.0  246.37  30.32  15.03  83.86
2024-01-12   9.67  33.62  441.49  11.75    1.0    1.0  247.33  30.80  15.26  84.72
2024-01-11   9.65  33.52  441.12  11.70    1.0    1.0  247.21  30.69  15.21  84.90
2024-01-10   9.61  33.36  441.40  11.65    1.0    1.0  247.36  30.67  15.22  85.46
2024-01-09   9.62  33.39  438.91  11.66    1.0    1.0  245.97  30.59  15.16  85.14
2024-01-08   9.63  33.67  439.50  11.68    1.0    1.0  246.37  30.85  15.29  86.12
2024

In [40]:
def last_friday():
    return datetime.date(2024, 1, 19)

def one_week_before(d1):
    return d1 - datetime.timedelta(days=7)

def pick_weekly_data(data, k=4):
    assert k <=6 and k >=0
    return data[list(map(lambda d: datetime.date.weekday(d)==k, data.index.tolist()))]

for tk in tickers:
    last_yr_close = histpx.loc[last_dt_prev_year][tk]
    last= last_friday()
    one_wk_before = one_week_before(last)
    last_px = histpx.loc[last][tk]
    prev_weekly_px =histpx.loc[one_wk_before][tk]
    onewk_earlier = one_week_before(last)
    wk_ret = (last_px-prev_weekly_px)/prev_weekly_px*100
    ytd_ret = (last_px-last_yr_close)/last_yr_close*100
    print(f"{round(wk_ret, 2)} (%),  {round(ytd_ret, 2)} ", tk, tk_2_name_d[tk])
    #print(tk, histpx[tk].head())

1.15 (%),  1.23  VTCLX Vanguard Tax-Managed Capital Ap
-1.14 (%),  -1.54  VBTLX Vanguard Total Bond Market Inde
0.06 (%),  -3.57  VTMSX Vanguard Tax Managed Small Cap 
-0.68 (%),  -0.51  VIPSX Vanguard Inflation Protected Se
1.19 (%),  1.53  VFIAX Vanguard 500 Index Fd Admiral S
-1.66 (%),  -2.54  VTIAX Vanguard Total International St
0.0 (%),  0.0  VMRXX Vanguard Cash Reserves Federal 
-1.57 (%),  -2.47  VTMGX Vanguard Developed Markets Inde
0.0 (%),  0.0  VMSXX Vanguard Municipal Money Market
-1.78 (%),  -3.05  VEMAX Vanguard Emerging Markets Stock


In [42]:
def get_name(r):
    #print(r.name)
    fund_name = tk_2_name_d[r.name].replace("Vanguard ", "")
    return fund_name

price_dates = [last_dt_prev_year, one_wk_before, last]
prices = histpx.loc[price_dates].T
prices.loc[:, 'fund name'] = prices.apply(get_name, axis=1)
prices.loc[:, '1_wk (%)'] = (prices[last] - prices[one_wk_before])/prices[one_wk_before]*100
prices.loc[:, 'ytd (%)'] = (prices[last] - prices[last_dt_prev_year])/prices[last_dt_prev_year]*100

prices.sort_values('ytd (%)')

,2023-12-29,2024-01-12,2024-01-19,fund name,1_wk (%),ytd (%)
VTMSX,87.91,84.72,84.77,Tax Managed Small Cap,0.06,-3.57
VEMAX,34.06,33.62,33.02,Emerging Markets Stock,-1.78,-3.05
VTIAX,31.08,30.80,30.29,Total International St,-1.66,-2.54
VTMGX,15.40,15.26,15.02,Developed Markets Inde,-1.57,-2.47
VBTLX,9.71,9.67,9.56,Total Bond Market Inde,-1.14,-1.54
VIPSX,11.73,11.75,11.67,Inflation Protected Se,-0.68,-0.51
VMRXX,1.00,1.00,1.00,Cash Reserves Federal,0.00,0.00
VMSXX,1.00,1.00,1.00,Municipal Money Market,0.00,0.00
VTCLX,247.12,247.33,250.17,Tax-Managed Capital Ap,1.15,1.23
VFIAX,439.99,441.49,446.73,500 Index Fd Admiral S,1.19,1.53


In [ ]:
pick_weekly_data(histpx)